In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:36075")
client

Client Scheduler: tcp://127.0.0.1:36231 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 9 Cores: 72 Memory: 810.00 GB


In [2]:
import dask.dataframe as dd

df = dd.read_csv("../../../data/train/viznet/all.csv", keep_default_na=False, dtype=str)

In [ ]:
import csv
import nltk
import numpy as np

df = df.persist()

def ngrams(x, n):
    tokens = x.split()
    if len(tokens) >= n:
        return [" ".join(ngram) for ngram in nltk.ngrams(x.split(), n)]
    else:
        return tokens

for i in range(1, 5):
    df[f"{i}_gram"] = df["data"].apply(ngrams, meta=(f"{i}_gram", np.array([""])), n=i)
    ngram_df = df.explode(f"{i}_gram")
    value2count = ngram_df[f"{i}_gram"].value_counts()
    value2count.index.name = "data"
    value2count.to_frame("count").to_csv(
        f"../../../data/train/viznet/ngram/{i}_gram_token_count.csv",
        single_file=True,
        quoting=csv.QUOTE_ALL,
    )
    df = df.drop(f"{i}_gram", axis=1)